In [2]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Optenemos el path de todas las imagenes
imgPaths = glob('./Selected/*.jpeg')

In [7]:
allBusinessCard = pd.DataFrame(columns=['id','text'])

# recorremos todas los path de las imagenes
# tqdm me muestra el porcentaje de como se va procesando
for imgPath in  tqdm(imgPaths,desc='BusinessCard'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    
    # extract data and text, lo mismo que antes
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    
    # conf el porcentaje que indica si la data esta ok o no
    df['conf'] = df['conf'].astype(int)
    useFulData = df.query('conf >= 30')

    # Dataframe
    businessCard = pd.DataFrame()
    businessCard['text'] = useFulData['text']
    businessCard['id'] = filename
    
    # concatenation
    allBusinessCard = pd.concat((allBusinessCard,businessCard))

BusinessCard: 100%|███████████████████████████████████████████████████████| 293/293 [02:37<00:00,  1.85it/s]


In [15]:
allBusinessCard.to_csv('businessCard.csv',index=False)